# Cyclic ID Algorithm Tutorial

---

In this tutorial notebook, you'll see how to use the **cyclic ID algorithm** to identify causal queries in cyclic graphs that contain:

- **Feedback Loops**
- **Latent confounders** (unmeasured causes)
- **Both of these simultaneously** (consolidated districts)


We will work through three queries on the same example graph to build intuition for how this algorithm works, and how you can use it for different graphs and other queries. 


## Why use the Cyclic ID Algorithm? 

Traditional causal inference algorithms like the **ID algorithm** assume graphs are **acyclic** (no feedback loops). However, many real systems contain cycles. For example, in biological systems, **gene regulatory networks** regulate each other which can cause many complex feedback loops and latent confounders. 

### The Challenge

When cycles are present in a graph, the traditional methods fail because:

1. There's no topological ordering of variables
2. Standard conditional independence relationships break down
3. Feedback loops create complex dependencies 

### One Solution: Cyclic ID Algorithm

The Cyclic ID algoritm (Forré & Mooij, 2019) extends causal identification to **input/output Structural Causal Models (ioSCMs)** that can handle:

- Directed cycles that form strongly connected components (SCCs).
- Latent confounders represented by bidirected edges.
- **Consolidated districts** where both cycles and confounders create complex structures.


Below we will see how this works with an example graph with genes. 

In [27]:
# Importing the necessary modules from the y0 library
from y0.dsl import A, B, C, D, P, E, X, Y, Z, R, V1
from y0.graph import NxMixedGraph
from y0.algorithm.identify import Unidentifiable, identify_outcomes
from y0.algorithm.identify.cyclic_id import cyclic_id

print("✓ Imports successful!")


✓ Imports successful!


### Graph Visualization

Below is the tutorial graph showing a cyclic graph that comprises of a feedback loop, a self-regulation loop and a latent confounder. 

![Tutorial Graph](./Graph_for_tutorial.png)

We will use this graph to create various queries that will either be identifiable or unidentifiable. You can do the same with a gene regulatory network for example. 

In [28]:
# Construct the tutorial graph with A, B, C, D, E
graph = NxMixedGraph.from_edges(
    directed=[
        (A, B),    
        (B, C),    
        (C, D),    
        (D, B),    # D → B: Completes feedback loop (B → C → D → B)
        (C, E),   
        (A, E),    
        (E, E),    # E → E: Self-loop 
    ],
    undirected=[
        (B, D),    # B ↔ D: Latent confounder
    ]
)

print("✓ Graph constructed successfully!")
print(f"Nodes: {sorted(graph.nodes(), key=str)}")
print(f"Directed edges: {len(graph.directed.edges())} edges")
print(f"Undirected edges: {len(graph.undirected.edges())} bidirected edge(s)")

✓ Graph constructed successfully!
Nodes: [A, B, C, D, E]
Directed edges: 7 edges
Undirected edges: 1 bidirected edge(s)


## Understanding the Graph Structure

### **What makes this graph interesting?**

This graph has three features that make traditional causal inference difficult:

1. **A feedback loop**: ℬ → 𝒞 → 𝒟 → ℬ (nodes affect each other in a cycle.)
2. **A self-loop**: ℰ → ℰ (the outcome feeds back to itself; common in networks)
3. **A latent confounder**: ℬ ↔ 𝒟 (shown as the red dashed bidirected line. - an unmeasured common cause)

These features create what's called a **consolidated district** - a group of nodes that are tightly connected through cycles and confounders. The cyclic ID algorithm is designed to handle exactly these situations.


## Exploring Different Queries

Using the graph we constructed above, we can now ask different causal questions to see when the cyclic ID algorithm can (and also can't) identify causal effects. 

We will look at **six queries** that demonstrate different scenarios:


- Some will end up being **identifiable** - we can compute them from observational data.
- Others will end up being **unidentifiable** - the algorithm will tell when possible, meaning other adjustments need to be made. 


Each query will help understand what is possible with cyclic causal inference.

### Query 1: Can we identify the effect of A on E?

The main question with this query is asking; what happens to the outcome (E) if we intervene on A? 

In causal inference, we can write this as: **P(E | do(A))**

### Why might this query work?

- A is external to the feedback loop
- There is a cycle between B, C, and D, and we are intervening from the outside. 
- There are multiple paths from A to E

Prediction: ✅ **Identifiable**

Below we see what the algorithm outputs with this exact query. 

In [29]:
# Query 1: P(E | do(A)) - Normal cycle query
print("=" * 60)
print("QUERY 1: P(E | do(A))")
print("=" * 60)

try:
    result_1 = cyclic_id(graph, outcomes={E}, interventions={A})
    print("\n✓ Result: ✅ IDENTIFIABLE")
    print(f"\nSymbolic expression:")
    print(f"{result_1}")
    print(f"\nSimplified:")
    print(f"{result_1.simplify()}")
except Unidentifiable as e:
    print("\n✗ Result: ❌ UNIDENTIFIABLE")
    print(f"Reason: {e}")

QUERY 1: P(E | do(A))

✓ Result: ✅ IDENTIFIABLE

Symbolic expression:
Sum[B, C, D]((P(A, B, C, D, E) / P(A)))

Simplified:
Sum[B, C, D]((P(A, B, C, D, E) / P(A)))


### What does this result mean? - Query 1

The algorithm successfully identified P(E | do (A)) as an identifiable causal query.

The symbolic expression tells us:

```
Sum[B, C, D]((P(A, B, C, D, E) / P(A)))
```

**Breaking this down:**
- We can compute the interventional distribution P(ℰ | do(𝒜)) from observational data
- The formula says: take the joint probability P(𝒜, ℬ, 𝒞, 𝒟, ℰ), divide by P(𝒜), then sum over ℬ, 𝒞, and 𝒟
- This marginalizes out the intermediate variables in the cycle, leaving just the effect on ℰ

**Key insight:** Even though there's a complex feedback loop and a latent confounder, we CAN identify this causal effect because we're intervening from **outside** the cycle.


### Query 2 - Can we identify the mediator variable? 

What happens to C (the mediator inside the cycle) if we intervene on A?

In causal inference notation: **P(C | do(A))**

### What makes this an interesting query?

- C is inside the consolidated district to identify.
- C is a part of the feedback loop. 
- We are still intervening from outside, but now asking about a node within the cycle. 

Prediction: **✅ Identifiable**

Since A is an external node, we should be able to trace the effect through to C, even though C is in the cycle.

In [30]:
# Query 2: P(C | do(A)) - Identifiable query (mediator in cycle)
print("=" * 60)
print("QUERY 2: P(C | do(A))")
print("=" * 60)

try:
    result_2 = cyclic_id(graph, outcomes={C}, interventions={A})
    print("\n✓ Result: ✅ IDENTIFIABLE")
    print(f"\nSymbolic expression:")
    print(f"{result_2}")
    print(f"\nSimplified:")
    print(f"{result_2.simplify()}")
except Unidentifiable as e:
    print("\n✗ Result: ❌ UNIDENTIFIABLE")
    print(f"Reason: {e}")

QUERY 2: P(C | do(A))

✓ Result: ✅ IDENTIFIABLE

Symbolic expression:
Sum[B, D]((P(A, B, C, D) / P(A)))

Simplified:
Sum[B, D]((P(A, B, C, D) / P(A)))


### What does this result mean? - Query 2

For the second query, the algorithm has successfully identified the query as identifiable, even though C is inside the cycle. 

The symbolic expression tells us:
```
Sum[B, D]((P(A, B, C, D) / P(A)))
```

**Breaking this down:**
- Take the joint probability P(𝒜, ℬ, 𝒞, 𝒟), divide by P(𝒜)
- Sum over ℬ and 𝒟 to marginalize them out
- This leaves us with the causal effect on 𝒞

**Key insight:** We can identify variables **within** the consolidated district {ℬ, 𝒞, 𝒟} as long as we're intervening from **outside** the district. The external intervention 𝒜 allows us to trace effects into the cycle.


## Query 3 - What about nodes that are confounded? 

What happens to D if we intervene on B? 

In causal inference notation, this is written as: **P(D | do(B))**

### Why might this not work?

- B and D have a **latent confounder**
- Both nodes are also a part of the same feedback loop, which means they are a part of the same district. 

### Rule of Thumb from Paper

From the paper: *"Causal effects where both cause and effect are inside the same strongly connected component (SCC) are generally not identifiable."*

Since B and D are both:
1. In the same SCC (feedback loop)
2. Connected by a latent confounder

Prediction: ❌ **This should be Unidentifiable.**

In [31]:
# Query 3: P(D | do(B)) - Latent confounding query
print("=" * 60)
print("QUERY 3: P(D | do(B))")
print("=" * 60)

try:
    result_3 = cyclic_id(graph, outcomes={D}, interventions={B})
    print("\n✓ Result: IDENTIFIABLE")
    print(f"\nSymbolic expression:")
    print(f"{result_3}")
    print(f"\nSimplified:")
    print(f"{result_3.simplify()}")
except Unidentifiable as e:
    print("\n✗ Result: ❌ UNIDENTIFIABLE)")
    print(f"Reason: {e}")

QUERY 3: P(D | do(B))

✗ Result: ❌ UNIDENTIFIABLE)
Reason: Cannot identify P{D} | do{B})). District frozenset({D}) failed identification.


## What does this result mean? - Query 3

As predicted, the algorithm cannot identify P(D | do (B)).


The error message tell us `"District frozenset {D} failed identification"`

Why did it fail identification?:
1. When we intervene on B (removing it from the graph), we need to identify the effect on D
2. But B and D are in the same **consolidated district** meaning they're merged by:
   - The feedback loop: B → C → D → B
   - The latent confounder: B ↔ D
3. The algorithm tries to "shrink" the district to isolate D, but finds that B (the intervention) is inseparable from D.

**Key insight:** When cause and effect are both inside the same consolidated district (same SCC + confounders), the causal effect is generally **not identifiable** from observational data itself. 

## Query 4: What about the reverse direction? 


What happens to B if we intervene on D? In casual inference notation, this is denoted as: **P(B | do(D))**.

### Why might this also fail? 

In Query 3, we saw that P(D | do(B)) was unidentifiable. Now if we ask the reverse question:
- B and D are still in the same consolidated district.
- They are still connected by a latent confounder.
- They are still in the same feedback loop.

Prediction: ❌ **This should also be unidentifiable.**


In [32]:
# Query 4: P(B | do(D)) - Non-identifiable query (reverse direction)
print("=" * 60)
print("QUERY 4: P(B | do(D))")
print("=" * 60)

try:
    result_4 = cyclic_id(graph, outcomes={B}, interventions={D})
    print("\n✓ Result: IDENTIFIABLE")
    print(f"\nSymbolic expression:")
    print(f"{result_4}")
    print(f"\nSimplified:")
    print(f"{result_4.simplify()}")
except Unidentifiable as e:
    print("\n✗ Result: ❌ UNIDENTIFIABLE.")
    print(f"Reason: {e}")

QUERY 4: P(B | do(D))

✗ Result: ❌ UNIDENTIFIABLE.
Reason: Cannot identify P{B} | do{D})). District frozenset({B}) failed identification.


### What does this result mean? - Query 4 

As expected, the algorithm cannot identify P(B | do(D)) either.

**Why did this fail?**

Just like Query 3, the consolidated district prevents identification:
- B and D are linked by the feedback loop and latent confounder.
- When we intervene on D, we can't isolate the causal effect on B.
- The algorithm correctly detects: `"District frozenset({B}) failed identification"`

**Key insight:** The consolidated district creates **symmetric unidentifiability** meaning if P(D | do(B)) is unidentifiable, then P(B | do(D)) is also unidentifiable. The causal entanglement works in both directions.

## Query 5 - Validation with Standard ID algorithm

So far we have seen various queries with a complex graph with cycles and confounders. But we also need to show how the cyclic ID algorithm works with simple acyclic graphs.

We can validate this by comparing the cyclic ID algorithm and the standard ID algorithm on a simple acyclic graph. Both algorithms should give the same answer, confirming that the cyclic ID algorithm reduces to the standard ID when no cycles are present. 

Below we create a basic causal chain: **A -> B -> C** 

In terms of causal inference, our query will be **P(C | do(A))**.

Both algorithms should return the same result as identifiable.

## Graph Visualization for Query 5

![Acyclic Graph](./Acyclic_graph.png)


In [33]:
# Create a simple acyclic graph for validation: A → B → C
simple_graph = NxMixedGraph.from_edges(
    directed=[
        (A, B),    # A → B
        (B, C),    # B → C
    ],
    undirected=[]  # No confounders
)

print("✓ Simple validation graph created!")
print(f"Nodes: {sorted(simple_graph.nodes(), key=str)}")
print(f"Edges: {list(simple_graph.directed.edges())}")
print("\nStructure: A → B → C (simple causal chain)")

✓ Simple validation graph created!
Nodes: [A, B, C]
Edges: [(A, B), (B, C)]

Structure: A → B → C (simple causal chain)


In [34]:
# Query 5: P(C | do(A)) on the simple acyclic graph
print("=" * 60)
print("QUERY 5: P(C | do(A)) on simple graph")
print("=" * 60)

try:
    result_5_cyclic = cyclic_id(simple_graph, outcomes={C}, interventions={A})
    print("\n✓ Cyclic ID Result: IDENTIFIABLE")
    print(f"\nSymbolic expression:")
    print(f"{result_5_cyclic}")
    print(f"\nSimplified:")
    print(f"{result_5_cyclic.simplify()}")
except Unidentifiable as e:
    print("\n✗ Cyclic ID Result: UNIDENTIFIABLE")
    print(f"Reason: {e}")

QUERY 5: P(C | do(A)) on simple graph

✓ Cyclic ID Result: IDENTIFIABLE

Symbolic expression:
Sum[B]((P(A, B, C) / P(A)))

Simplified:
Sum[B]((P(A, B, C) / P(A)))


In [35]:
# Now run Standard ID on the same query
print("\n" + "=" * 60)
print("STANDARD ID for comparison:")
print("=" * 60)

try:
    result_5_standard = identify_outcomes(simple_graph, outcomes={C}, treatments={A})
    print("\n✓ Standard ID Result: IDENTIFIABLE")
    print(f"\nSymbolic expression:")
    print(f"{result_5_standard}")
    print(f"\nSimplified:")
    print(f"{result_5_standard.simplify()}")
except Exception as e:
    print("\n✗ Standard ID Result: ❌ Failed")
    print(f"Reason: {e}")


STANDARD ID for comparison:

✓ Standard ID Result: IDENTIFIABLE

Symbolic expression:
Sum[B](P(B | A) * P(C | A, B))

Simplified:
Sum[B](P(B | A) * P(C | A, B))


### Comparing the Results for Query 5

Both algorithms have successfull identified the query as identifiable for the acyclic graph. 

**Cyclic ID gives:**
```
Sum[B]((P(A, B, C) / P(A)))
```

**Standard ID gives:**
```
Sum[B](P(B | A) * P(C | A, B))
```

**Are these the same?** They are the same, just written in different forms:
- **Cyclic ID** uses the fraction form: `P(A, B, C) / P(A)`
- **Standard ID** uses the conditional form: `P(B | A) * P(C | A, B)`

Using the chain rule of probability, we can show they're equal:
- P(B | A) = P(A, B) / P(A)
- P(C | A, B) = P(A, B, C) / P(A, B)
- Multiply: P(B | A) * P(C | A, B) = P(A, B, C) / P(A)

✅ **Validation successful:** Cyclic ID correctly reduces to standard ID on acyclic graphs.

## Query 6 - Intervening on a node with a self-loop

What happens to other nodes when we intervene on E, which has a self-loop? Refer back to the original graph in which E has a self-loop: E → E (it feeds back to itself).

The query for this example will be: **P(C | do(E))

## What does this mean?
- We're intervening on E, which **removes** the self-loop
- We're asking about C, which is **upstream** in the causal graph
- Path from C to E : C → E (direct)

### What happens to the self-loop?

When we intervene on E (do(E)):
1. All edges **pointing into** E are removed (including E → E)
2. The self-loop is **broken** by the intervention
3. E is now fixed, no longer influenced by itself or its ancestors.

Prediction: This could go either way. 

In [36]:
# Query 6: P(C | do(E)) - Self-loop query
print("=" * 60)
print("QUERY 6: P(C | do(E))")
print("=" * 60)
print("Note: E has a self-loop (E → E) which gets removed by the intervention")
print()

try:
    result_6 = cyclic_id(graph, outcomes={C}, interventions={E})
    print("\n✓ Result: ✅ IDENTIFIABLE")
    print(f"\nSymbolic expression:")
    print(f"{result_6}")
    print(f"\nSimplified:")
    print(f"{result_6.simplify()}")
except Unidentifiable as e:
    print("\n✗ Result: ❌ UNIDENTIFIABLE")
    print(f"Reason: {e}")

QUERY 6: P(C | do(E))
Note: E has a self-loop (E → E) which gets removed by the intervention


✓ Result: ✅ IDENTIFIABLE

Symbolic expression:
Sum[A, B, D](P(A, B, C, D))

Simplified:
P(C)


### What does this mean? - Query 6

The result ends up being identifiable, and simpliefies to just **P(C)**.

**The symbolic expression:**
```
Sum[A, B, D](P(A, B, C, D)) = P(C)
```

**This is marginalizing the joint distribution to get P(C).**

**What does P(C | do(E)) = P(C) mean?**

This result tells us about causal direction:
- **Intervening on E has no effect on C**
- Because there is **no causal path from C to C** in our graph
- The only connection is C → E (C causes E, not the other way around)
- Therefore: fixing E doesn't change the distribution of C

This demonstrates **Rule 3 of causal calculus** (Insertion/Deletion of Actions): when the outcome is independent of the intervention, P(Y | do(X)) = P(Y).

**What about the self-loop?**
- When we intervene on E, the self-loop E -> E is **removed** (graph mutilation).
- All edges pointing into E are deleted, including the self-loop.
- The algorithm correctly handles this structural change.

**Pedagogical value:**

This query demonstrates three important concepts:
1. **Graph mutilation** - interventions remove incoming edges (including self-loops)
2. **Null causal effects** - the algorithm correctly identifies when an intervention has no effect
3. **Marginalization** - how the algorithm uses ancestral distributions to compute outcomes

**Technical note:** The self-loop E → E is treated as a single-node Strongly Connected Component (SCC) - the simplest form of a feedback loop. When the algorithm processes districts, it handles this 1-node SCC just like it handles the larger 3-node SCC {B, C, D}. (See explanation notebook for more information.)
